In [1]:
from __future__ import division
from __future__ import print_function

import time
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric

from  utils import run_experiment
from models import GCN, iterativeGCN
from torch_geometric.datasets import Planetoid

In [2]:
from torch_geometric.transforms import NormalizeFeatures
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())

In [3]:
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("Training sample: ", data.train_mask.sum().item())
print("Validation sample: ", data.val_mask.sum().item())
print("Test sample: ", data.test_mask.sum().item())
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Training sample:  140
Validation sample:  500
Test sample:  1000
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Has isolated nodes: False
Has self-loops: False
Is undirected: True


In [ ]:
LR = np.arange(0.001, 0.01, 0.0005)
SMOOTH_FAC = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
curr_exp = 1
for lr in LR:
    for smooth_fac in SMOOTH_FAC:
        model = iterativeGCN(input_dim=dataset.num_features,
                                output_dim=dataset.num_classes,
                                hidden_dim=32,
                                num_train_iter=2,
                                num_eval_iter=2,
                                smooth_fac=smooth_fac,
                                dropout=0.5)
        loss_test, acc_test, training_time = run_experiment(model=model, 
                                                            data=data, 
                                                            lr=lr, 
                                                            weight_decay=2e-4,
                                                            model_name=str(LR) + "," + str(smooth_fac),
                                                            run=1,
                                                            num_epochs=200)
        print("Experiment ", curr_exp, ", training time: ", training_time)
        print("Learning rate: ", lr, ", smoothing factor: ", smooth_fac, " loss: ", loss_test, ", accuracy: ", acc_test)
        curr_exp += 1
        del model

In [ ]:
two_layer_GCN = GCN(input_dim=dataset.num_features,
                    output_dim=dataset.num_classes,
                    hidden_dim=16,
                    num_layers=2,
                    dropout=0.5)
run_experiment(model=two_layer_GCN,
               data=data,
               lr=0.01,
               weight_decay=5e-4,
               model_name="2l",
               run=1,
               num_epochs=200)

In [ ]:
print(two_layer_GCN)

In [6]:
feature_dim = dataset.num_features
num_class = dataset.num_classes
dropout = 0.5
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self, hidden_dim, feature_dim, num_class, dropout=0.5) -> None:
        super().__init__()
        self.gc1 = GCNConv(feature_dim, hidden_dim)
        self.gc2 = GCNConv(hidden_dim, num_class)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = F.relu(self.gc1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.gc2(x, edge_index)
        x = F.log_softmax(x, dim=1)
        
        return x
    
model22 = GCN(hidden_dim=16,
            feature_dim=feature_dim,
            num_class=num_class,
            dropout=0.5
            )
print(model22)

run_experiment(model=model22,
               data=data,
               lr=0.01,
               weight_decay=5e-4,
               model_name="2l",
               run=1,
               num_epochs=200)

GCN(
  (gc1): GCNConv(1433, 16)
  (gc2): GCNConv(16, 7)
)
Epoch: 0001 loss_train: 1.9448 acc_train: 0.1357 loss_val: 1.9432 acc_val: 0.1240 time: 0.0165s
Epoch: 0002 loss_train: 1.9374 acc_train: 0.3571 loss_val: 1.9364 acc_val: 0.2980 time: 0.0115s
Epoch: 0003 loss_train: 1.9307 acc_train: 0.4500 loss_val: 1.9296 acc_val: 0.4920 time: 0.0106s
Epoch: 0004 loss_train: 1.9133 acc_train: 0.6571 loss_val: 1.9238 acc_val: 0.4400 time: 0.0109s
Epoch: 0005 loss_train: 1.9073 acc_train: 0.5286 loss_val: 1.9185 acc_val: 0.4360 time: 0.0105s
Epoch: 0006 loss_train: 1.8958 acc_train: 0.5929 loss_val: 1.9130 acc_val: 0.4360 time: 0.0119s
Epoch: 0007 loss_train: 1.8761 acc_train: 0.6000 loss_val: 1.9073 acc_val: 0.4280 time: 0.0119s
Epoch: 0008 loss_train: 1.8616 acc_train: 0.6357 loss_val: 1.9009 acc_val: 0.4300 time: 0.0127s
Epoch: 0009 loss_train: 1.8550 acc_train: 0.6071 loss_val: 1.8941 acc_val: 0.4340 time: 0.0126s
Epoch: 0010 loss_train: 1.8395 acc_train: 0.6214 loss_val: 1.8864 acc_val: 0.4

(0.7862623929977417, 0.805, 2.441358804702759)